In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from constants import DatasetPath

In [2]:
#Utility constants
DATASET_PATH = DatasetPath.effectivePath

def DATASET_DIRS():
    return os.listdir(DATASET_PATH)

csv_columns_name = ['filename', 'image_path', 'target', 'category']


DIR_FOURIER_PATH = DATASET_PATH + "fourier\\"
FFTS_PATH = DIR_FOURIER_PATH + "fourier\\"

In [3]:
if("fourier" in DATASET_DIRS()):
    print("Fourier folder already exist.")
else:
    mode = 0o666
    path = os.path.join(DATASET_PATH,"fourier")
    #creates the fourier folder in the main Dataset folder
    os.mkdir(path,mode)
    
    #creates a fourier folder in the fourier folder
    os.mkdir(path + "\\fourier", mode)

In [4]:
#APPLICO LA TRASFORMATA DI FOURIER SUL TINY DATASET

fourier_csv = pd.DataFrame(columns = csv_columns_name)

with open(DATASET_PATH  + "\\trainingSet.csv", mode='r', newline='') as train_set:
    train_set_csv = pd.read_csv(train_set)
    
    for index, row in tqdm(train_set_csv.iterrows()):
        
        path = DATASET_PATH + row["image_path"]
    
        RGBimg = cv2.imread(path)
        grayImg = cv2.cvtColor(RGBimg, cv2.COLOR_BGR2GRAY)
        
        fft_img = np.fft.fft2(grayImg)
        fft_img = np.log(np.abs(fft_img))

        min_val = np.min(fft_img)
        max_val = np.max(fft_img)
        
        #fft_img = cv2.normalize(fft_img, None, 0, 255, cv2.NORM_MINMAX)
        fft_img = (fft_img - min_val) * (255.0 / (max_val - min_val))
        fft_img = np.uint8(fft_img)     
        
        filename = row["filename"]
        if row["target"] == 0:
            filename = filename.replace("img","real")
        else:
            filename = filename.replace("img","fake")

        fft_path = FFTS_PATH + filename
        cv2.imwrite(fft_path, fft_img)
        
        split_fft_path = fft_path.split(DATASET_PATH)[-1]
        fourier_csv.loc[len(fourier_csv)] = [filename,split_fft_path,row["target"],row["category"]]
    
    fourier_csv.to_csv(DIR_FOURIER_PATH + "metadata.csv", index=False)
    
    #plt.imshow(fft_img)
    #plt.show()

0it [00:00, ?it/s]

In [16]:
input_df = pd.DataFrame(columns= ["anchor","positive","negative"])

with open(DIR_FOURIER_PATH + "metadata.csv", mode='r', newline='') as fourier_csv:
    fourier_df = pd.read_csv(fourier_csv)
    input_df["anchor"] = fourier_df["image_path"]

    real_fourier_df = fourier_df[ fourier_df["target"] == 0 ]
    fake_fourier_df = fourier_df[ fourier_df["target"] != 0 ]
    
    
    for index, row in input_df.iterrows():
        image_path = row["anchor"]
        
        if "real" in image_path:
            positive_img = real_fourier_df.sample(frac=1).head(1)
            while positive_img["image_path"].iloc[0] == image_path:
                positive_img = real_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "positive"] = positive_img["image_path"].iloc[0]
            negative_img = fake_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "negative"] = negative_img["image_path"].iloc[0]
        elif "fake" in image_path:
            positive_img = real_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "negative"] = positive_img["image_path"].iloc[0]
            negative_img = fake_fourier_df.sample(frac=1).head(1)
            while negative_img["image_path"].iloc[0] == image_path:
                negative_img = fake_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "positive"] = negative_img["image_path"].iloc[0]
            

input_df.to_csv(DATASET_PATH + "input.csv", index=False)